# A Test for the Datashader Application

This uses [importnb](https://pypi.org/project/importnb) to run the
[Datashader_Example](./Datashader_Example.ipynb) notebook.

In [ ]:
import json
import os
import time
from pathlib import Path

import importnb
from rdflib import Graph

from ipyradiant import FileManager, PathLoader

with importnb.Notebook():
    try:
        from .Datashader_Example import T, W, ds, ds_ex, ls, sn
    except ImportError:
        from Datashader_Example import T, W, ds, ds_ex, ls, sn

This notebook is used as part of the `ipyradiant` test suite, where `IPYRADIANT_TESTING`
will be set, which will trigger the automted section below.

In [ ]:
IS_TESTING = json.loads(os.environ.get("IPYRADIANT_TESTING", "false"))
IS_TESTING

Tests are implemented as buttons you can click.

In [ ]:
pl = PathLoader(path="data")
fm = FileManager(loader=pl)
T.link((pl, "graph"), (ds, "graph"))

In [ ]:
tests = [W.Button(description=f"{k}") for k, v in pl.file_picker.options.items() if v]

In [ ]:
timings = {}


def timestamp(key, msg):
    if key not in timings:
        timings[key] = []
    timings[key] += [time.time()]
    delta = 0 if len(timings[key]) == 1 else timings[key][-1] - timings[key][-2]
    print(f"[{key}]", f"+{int(delta)}", msg)

In [ ]:
def _run_test(btn):
    p = btn.description

    try:
        timestamp(p, "starting...")
        fm.graph = Graph()
        timestamp(p, "cleaned...")
        assert ds.output_graph == None
        timestamp(p, f"loading...")
        pl.file_picker.value = pl.file_picker.options[p]
        timestamp(p, f"... {len(fm.graph)} triples loaded")
        assert len(ds.output_graph) > 0
        timestamp(p, "OK!")
    except Exception as err:
        timestamp(p, "ERROR")
        timestamp(p, err)
        raise Exception(f"{p} failed") from err


[d.on_click(_run_test) for d in tests]

# Show the test application

In [ ]:
W.VBox([ds_ex, W.HBox([W.Label("Start Test"), *tests])])

## (Maybe) Run the test automatically

In [ ]:
if IS_TESTING:
    for test in tests:
        test.click()